In [4]:
import pandas as pd

csv_file = 'books .csv'
try:
    data = pd.read_csv(csv_file) # Use on_bad_lines='skip'

except Exception as e:
    print(f"An error occurred: {e}")

# You can optionally check how many lines were skipped if you use a logging handler
# or if pandas provides a warning. With 'skip', it usually just works without explicit count in output.

# Display the first few rows to verify
print(data.tail(30))
data.to_csv('books2.csv')

                                       bookId  \
52448                         1126663.Charmed   
52449                   243854.Vacuum_Flowers   
52450                            13501282-gio   
52451             17333265-mr-tiger-goes-wild   
52452                    37169.The_Delta_Star   
52453                    15534.Question_Quest   
52454                  13254398-au-petit-poil   
52455                         12139342-archon   
52456              2088519.Sex_at_the_Margins   
52457               17977864-the-long-weekend   
52458                 2147003.Mission_to_Asia   
52459           797498.The_Last_Three_Minutes   
52460                2037584.To_Hold_Infinity   
52461               22925176-sweet-possession   
52462                                 2613071   
52463      25876358-the-natural-way-of-things   
52464                      36374396-algedonic   
52465                   6403092-arafel-s-saga   
52466  8756401-theodosia-and-the-last-pharaoh   
52467               

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52478 entries, 0 to 52477
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            52478 non-null  object 
 1   title             52478 non-null  object 
 2   series            23470 non-null  object 
 3   author            52478 non-null  object 
 4   rating            52478 non-null  float64
 5   description       51140 non-null  object 
 6   language          48672 non-null  object 
 7   isbn              52478 non-null  object 
 8   genres            52478 non-null  object 
 9   characters        52478 non-null  object 
 10  bookFormat        51005 non-null  object 
 11  edition           4955 non-null   object 
 12  pages             50131 non-null  object 
 13  publisher         48782 non-null  object 
 14  publishDate       51598 non-null  object 
 15  firstPublishDate  31152 non-null  object 
 16  awards            52478 non-null  object

In [ ]:
import os
import pandas as pd
import kagglehub

# Download and get the dataset path
dataset_path = kagglehub.dataset_download("snehangsude/audible-dataset")

# Find all files in the dataset folder
files = os.listdir(dataset_path)
print("Files in the dataset:", files)

# Find the first CSV file in the dataset
csv_file = [file for file in files if file.endswith(".csv")][0]
csv_path = os.path.join(dataset_path, csv_file)

# Load the CSV file into a Pandas DataFrame
data = pd.read_csv(csv_path)

# Display the first few rows
print("Dataset preview:")
print(data.head())

# Save the DataFrame to a new CSV file
output_path = os.path.join("audio_book.csv")
data.to_csv(output_path, index=False)
print(f"Saved as: {output_path}")


Files in the dataset: ['audio_book.csv', 'audible_uncleaned.csv', 'audible_cleaned.csv']
Dataset preview:
                                         name                     author  \
0                  Geronimo Stilton #11 & #12  Writtenby:GeronimoStilton   
1                            The Burning Maze      Writtenby:RickRiordan   
2                                The Deep End       Writtenby:JeffKinney   
3                        Daughter of the Deep      Writtenby:RickRiordan   
4  The Lightning Thief: Percy Jackson, Book 1      Writtenby:RickRiordan   

                    narrator                time releasedate language  \
0      Narratedby:BillLobely   2 hrs and 20 mins    04-08-08  English   
1   Narratedby:RobbieDaymond   13 hrs and 8 mins    01-05-18  English   
2      Narratedby:DanRussell    2 hrs and 3 mins    06-11-20  English   
3  Narratedby:SoneelaNankani  11 hrs and 16 mins    05-10-21  English   
4  Narratedby:JesseBernstein              10 hrs    13-01-10  English   

In [10]:
import streamlit as st
import pandas as pd
import urllib.parse

# mock dataset
data = [
    {
        "title": "Geronimo Stilton #11 & #12",
        "author": "Geronimo Stilton",
        "narrator": "Bill Lobely",
        "time": "2 hrs and 20 mins",
        "releasedate": "04-08-08",
        "language": "English",
        "stars": 5.0,
        "votes": 34,
        "price": 468.0,
    },
    {
        "title": "The Burning Maze",
        "author": "Rick Riordan",
        "narrator": "Robbie Daymond",
        "time": "13 hrs and 8 mins",
        "releasedate": "01-05-18",
        "language": "English",
        "stars": 4.5,
        "votes": 41,
        "price": 820.0,
    },
    # Add more entries here...
]

df = pd.DataFrame(data)

# utility to generate Audible link
def generate_audible_link(title):
    base_url = "https://www.audible.in/search"
    query = urllib.parse.urlencode({"keywords": title, "k": title})
    return f"{base_url}?{query}"

# streamlit UI
st.title("Audible Dataset Explorer")

# Filters
st.sidebar.header("Filters")
min_price, max_price = st.sidebar.slider("Price Range", 0, int(df["price"].max()), (0, int(df["price"].max())))
min_votes = st.sidebar.slider("Minimum Votes", 0, int(df["votes"].max()), 0)
min_stars = st.sidebar.slider("Minimum Stars", 0.0, 5.0, 0.0, step=0.5)

# Apply filters
filtered_df = df[
    (df["price"] >= min_price) &
    (df["price"] <= max_price) &
    (df["votes"] >= min_votes) &
    (df["stars"] >= min_stars)
]

# Sorting
sort_by = st.sidebar.selectbox("Sort By", ["stars", "votes", "price"])
sort_order = st.sidebar.radio("Sort Order", ["Ascending", "Descending"])
filtered_df = filtered_df.sort_values(by=sort_by, ascending=(sort_order == "Ascending"))

# Display Data
st.write(f"Showing {len(filtered_df)} results:")
for _, row in filtered_df.iterrows():
    st.markdown(f"### {row['title']}")
    st.write(f"**Author:** {row['author']}")
    st.write(f"**Narrator:** {row['narrator']}")
    st.write(f"**Time:** {row['time']}")
    st.write(f"**Release Date:** {row['releasedate']}")
    st.write(f"**Language:** {row['language']}")
    st.write(f"**Stars:** {row['stars']}")
    st.write(f"**Votes:** {row['votes']}")
    st.write(f"**Price:** ₹{row['price']}")
    st.markdown(f"[Search on Audible]({generate_audible_link(row['title'])})")
    st.markdown("---")


2025-05-09 02:52:57.675 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 02:52:57.747 
  command:

    streamlit run /home/arch/.venvs/global/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-05-09 02:52:57.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 02:52:57.750 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 02:52:57.751 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 02:52:57.760 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 02:52:57.762 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-09 02:52:57.763 Thread 'Main

In [3]:
import re

# Create DataFrame
df = pd.DataFrame(data)

# Extract numerical ratings and votes
def parse_stars_and_votes(stars):
    rating_match = re.search(r"([\d.]+) out of 5 stars", stars)
    votes_match = re.search(r"(\d+) ratings", stars)
    rating = float(rating_match.group(1)) if rating_match else None
    votes = int(votes_match.group(1)) if votes_match else 0
    return rating, votes

df[["rating", "votes"]] = df["stars"].apply(parse_stars_and_votes).apply(pd.Series)

# Set threshold and compute weighted score
m = 30  # Minimum votes threshold
c = df["rating"].mean()  # Mean rating across all items

df["weighted_score"] = ((df["votes"] * df["rating"]) + (m * c)) / (df["votes"] + m)

# Sort by weighted score
df = df.sort_values(by="weighted_score", ascending=False)

# Filter items below threshold votes
threshold = 10
df = df[df["votes"] >= threshold]

# Display results
print("Ranked Items:")
print(df[["name", "rating", "votes", "weighted_score"]])



Ranked Items:
                                                  name  rating  votes  \
44433                                Project Hail Mary     5.0  926.0   
7559   Harry Potter and the Chamber of Secrets, Book 2     5.0  845.0   
48143       Sherlock Holmes: The Definitive Collection     5.0  838.0   
64428                                    Wings of Fire     5.0  827.0   
22158                                    Wings of Fire     5.0  823.0   
...                                                ...     ...    ...   
66887                                         Hinduism     3.0   24.0   
16691                                Unf--k Your Brain     3.5   85.0   
19433                                       Just Do It     3.5  112.0   
60826                                The Sands of Time     3.5  134.0   
12490                    Learn English: Word Power 101     3.0   82.0   

       weighted_score  
44433        4.982958  
7559         4.981381  
48143        4.981231  
64428        